In [21]:
!pip install pyngrok


In [22]:
!pip install flask-cors

In [23]:
!ngrok authtoken 2poaD6Yusc9t15CgEkRtu51afUT_GuRE4YKPpYifDS22drks


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import flask_cors
from pyngrok import ngrok
import joblib
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Membuat aplikasi Flask
app = Flask(__name__)

# Mengaktifkan CORS
CORS(app)

# Memuat model dan TF-IDF
def load_model():
    tfidf = joblib.load('tfidf_vectorizer.pkl')  # Ganti dengan lokasi file tfidf_vectorizer.pkl
    model = joblib.load('language_model.pkl')  # Ganti dengan lokasi file model.pkl
    return tfidf, model

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = text.lower()  # Mengubah semua huruf menjadi kecil
    text = re.sub(r'\s+', ' ', text)  # Menghapus whitespace berlebih
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter non-alfanumerik
    return text

# Endpoint untuk prediksi bahasa
@app.route('/predict', methods=['POST'])
def predict_language():
    try:
        # Memuat model
        tfidf, model = load_model()

        # Menerima data input dari pengguna
        data = request.get_json()
        user_text = data.get('text', '')

        if not user_text:
            return jsonify({'error': 'Text is required'}), 400

        # Pembersihan teks
        cleaned_text = clean_text(user_text)

        # Transformasi teks dengan TF-IDF
        text_tfidf = tfidf.transform([cleaned_text])

        # Prediksi bahasa
        predicted_lang = model.predict(text_tfidf)

        # Pastikan prediksi mengembalikan hasil yang valid
        if predicted_lang is not None:
            return jsonify({'predicted_language': predicted_lang[0]})
        else:
            return jsonify({'error': 'Prediction failed'}), 500

    except Exception as e:
        # Menangani error jika terjadi
        print(f"Error: {str(e)}")  # Menambahkan log untuk melihat error
        return jsonify({'error': str(e)}), 500


# Menjalankan aplikasi Flask
if __name__ == '__main__':
    # Buka ngrok tunnel
    public_url = ngrok.connect(5000)  # Tidak perlu menambahkan `use_https=True`
    print(f"[*] Flask app ngrok tunnel \"{public_url}\"")
    app.run(host='0.0.0.0', port=5000)


[*] Flask app ngrok tunnel "NgrokTunnel: "https://1b1c-34-105-115-77.ngrok-free.app" -> "http://localhost:5000""
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:12:31] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:13:17] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:13:17] "POST /predict HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:14:11] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:14:12] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:26:33] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:26:34] "POST /predict HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:26:49] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Dec/2024 02:29:39] "OPT